In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/drive/MyDrive/RuSportSum/ods_broadcasts_201905301157.csv.zip" /content/ods_broadcasts_201905301157.csv.zip
!cp "/content/drive/MyDrive/RuSportSum/resul_data.csv" /content/resul_data.csv
!unzip /content/ods_broadcasts_201905301157.csv.zip

Archive:  /content/ods_broadcasts_201905301157.csv.zip
  inflating: ods_broadcasts_201905301157.csv  


In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup

import random
from pprint import pprint
from IPython.display import clear_output

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
data_broadcasts = pd.read_csv("/content/ods_broadcasts_201905301157.csv")
data_broadcasts

,match_id,team1,team2,name,match_time,type,minute,content,message_time
0,18925,Локомотив,Химки,Россия. Премьер-лига,1237037400,NaN,0,"Добрый день! Sports.ru поздравляет всех, кто п...",1237034763
1,18925,Локомотив,Химки,Россия. Премьер-лига,1237037400,NaN,0,"Будем надеяться, что признаков пресловутого ""в...",1237034996
2,18925,Локомотив,Химки,Россия. Премьер-лига,1237037400,NaN,0,"Что ж, сегодняшних соперников можно назвать од...",1237035202
3,18925,Локомотив,Химки,Россия. Премьер-лига,1237037400,NaN,0,Зимой подмосковный клуб едва не исчез с футбол...,1237035415
4,18925,Локомотив,Химки,Россия. Премьер-лига,1237037400,NaN,0,"""Локомотив"" зимой старался стать лучше. В сост...",1237035796
...,...,...,...,...,...,...,...,...,...
722062,1104866,Питтсбург,Сан-Хосе,Кубок Стэнли,1464825600,empty,63,"<b>2:1 - ""Питтсбург"" выигрывает второй финальн...",1464837758
722063,1104866,Питтсбург,Сан-Хосе,Кубок Стэнли,1464825600,empty,63,"Очевидно, что залог успеха ""Питтсбурга"" - иниц...",1464837879
722064,1104866,Питтсбург,Сан-Хосе,Кубок Стэнли,1464825600,empty,63,"Другими словами, ДеБур должен в первую очередь...",1464838029
722065,1104866,Питтсбург,Сан-Хосе,Кубок Стэнли,1464825600,empty,63,Большое спасибо за чтение! До новых встреч на ...,1464838220


In [ ]:
data_broadcasts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 722067 entries, 0 to 722066
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   match_id      722067 non-null  int64 
 1   team1         722067 non-null  object
 2   team2         722067 non-null  object
 3   name          722067 non-null  object
 4   match_time    722067 non-null  int64 
 5   type          542930 non-null  object
 6   minute        722067 non-null  int64 
 7   content       721636 non-null  object
 8   message_time  722067 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 49.6+ MB


In [ ]:
def remove_tags(html):
    soup = BeautifulSoup(str(html), "html.parser")

    for data in soup(['style', 'script']):
        data.decompose()

    return ' '.join(soup.stripped_strings)

data_broadcasts['content'] = data_broadcasts['content'].apply(remove_tags)

data_broadcasts['content'] = data_broadcasts['content'].apply(lambda x: re.sub(r'[Гг]+[Оо]+[Лл]+', 'гол', x))

<ipython-input-6-33f81898c48c>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(str(html), "html.parser")
<ipython-input-6-33f81898c48c>:2: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(str(html), "html.parser")


In [ ]:
def concatenate_content(messages):
  return ' '.join([str(message).strip() for message in messages])

def last_part_content(messages, size=200):
  result = ""
  for message in messages[::-1]:
    prep_message = message.strip()
    if len((result + prep_message).split()) < size:
      result = prep_message + " " + result
    else:
      break
  return result.strip()

grouping_columns = ['match_id', 'team1', 'team2', 'name', 'match_time']
data_broadcasts = data_broadcasts.sort_values(['match_id', 'minute', 'message_time'], ascending = True).groupby(grouping_columns)['content'].agg([concatenate_content, last_part_content]).reset_index()
data_broadcasts

,match_id,team1,team2,name,match_time,concatenate_content,last_part_content
0,18925,Локомотив,Химки,Россия. Премьер-лига,1237037400,"Добрый день! Sports.ru поздравляет всех, кто п...",Драган Блатняк запустил мяч на трибуну со штра...
1,18935,Зенит,Штутгарт,перенесенные матчи,1234977300,"Матчем со ""Штутгартом"" ""Зенит"" открывает европ...",Шибер впереди пока никак не зацепится за мяч. ...
2,18936,Астон Вилла,ЦСКА,перенесенные матчи,1234983600,"Интересный парадокс жребия - армейцам, единств...",Люк Янг заработал на Карвальо желтую карточку!...
3,18941,Шахтер,Тоттенхэм,перенесенные матчи,1235069100,Приветствуем Всех любителей самой популярной и...,"По количеству нарушений, можно сказать, что фу..."
4,18956,Штутгарт,Зенит,перенесенные матчи,1235677500,"Ответный матч 1/16 финала Кубка УЕФА ""Зенита"" ...",Гомес толкнул Крижанаца в верховой борьбе. Нап...
...,...,...,...,...,...,...,...
8776,1104438,Томь,Кубань,Россия. Премьер-лига,1464354000,Привет. Теперь точно – сегодня последний тур Р...,Самое ужасное в этой ситуации – потенциальная ...
8777,1104439,Волгарь,Анжи,Россия. Премьер-лига,1464102000,Всем привет! Стыковые матчи за право выступать...,"Алейник уступает место Петровичу, известному п..."
8778,1104440,Анжи,Волгарь,Россия. Премьер-лига,1464364800,Всем привет! Сегодня важный день для российско...,В опасном подкате Зуйков сбивает с ног Маевско...
8779,1104865,Питтсбург,Сан-Хосе,Кубок Стэнли,1464652800,"Привет всем, кто не представляет своей жизни б...","""Сан Хосе"" не помогает и замена вратаря. Плюс ..."


In [ ]:
data_news = pd.read_csv("/content/resul_data.csv")

data_news = data_news[data_news['relevance'] == 1].drop(['relevance'], axis=1)
data_news

,id,name,ctime,body,match_id,name_body
0,70098967,Денисов получил травму в матче с «Локомотивом»,2010-04-11 15:35:05,Полузащитник «Зенита» Игорь Денисов получил по...,32552,Денисов получил травму в матче с «Локомотивом»...
1,1035039233,«Анжи» – ЦСКА – 1:1. Максимов спас хозяев от п...,2015-11-29 09:28:46,Сегодня в 17-м туре чемпионата России ЦСКА на ...,1035349,«Анжи» – ЦСКА – 1:1. Максимов спас хозяев от п...
2,149063708,«Чикаго» – «Детройт». Кэйн забросил первую шай...,2013-05-18 21:36:16,Форвард «Чикаго» Патрик Кэйн открыл счет во вт...,780500,«Чикаго» – «Детройт». Кэйн забросил первую шай...
3,157992899,Финляндия – Россия. Бобровский заменил Варламо...,2014-02-19 17:39:35,По ходу второго периода матча 1/4 финала олимп...,829733,Финляндия – Россия. Бобровский заменил Варламо...
4,1027781272,Роббен одержал 100-ю победу в бундеслиге и уст...,2015-03-07 19:30:14,В матче 24-го тура чемпионата Германии «Бавари...,965242,Роббен одержал 100-ю победу в бундеслиге и уст...
...,...,...,...,...,...,...
24640,1040621229,НХЛ. Кубок Стэнли. Финал. «Питтсбург» вновь по...,2016-06-01 18:17:21,Во втором матче финала Кубка Стэнли «Питтсбург...,1104866,НХЛ. Кубок Стэнли. Финал. «Питтсбург» вновь по...
24642,1040625581,Нойштедтер дебютировал за сборную России,2016-06-01 20:23:14,Новобранец сборной России Роман Нойштедтер деб...,1095139,Нойштедтер дебютировал за сборную России. Ново...
24644,1040636713,НХЛ. Форвард «Питтсбурга» Шери забил первый по...,2016-06-02 08:20:10,Во втором матче финала Кубка Стэнли «Питтсбург...,1104866,НХЛ. Форвард «Питтсбурга» Шери забил первый по...
24645,1040636981,НХЛ. Мюррей одержал 13-ю победу в дебютном пле...,2016-06-02 08:41:16,Во втором матче финала Кубка Стэнли «Питтсбург...,1104866,НХЛ. Мюррей одержал 13-ю победу в дебютном пле...


In [ ]:
data_news['len_name_body'] = data_news['name_body'].apply(lambda x: len(x.split()))
data_news.describe()

,id,match_id,len_name_body
count,1.283800e+04,1.283800e+04,12838.000000
mean,5.125260e+08,7.169777e+05,60.644882
std,4.455023e+08,3.005521e+05,24.378935
min,7.014478e+06,1.892500e+04,7.000000
25%,1.417840e+08,4.784990e+05,43.000000
50%,1.546640e+08,7.872220e+05,57.000000
75%,1.027417e+09,9.820148e+05,73.000000
max,1.040637e+09,1.104866e+06,400.000000


In [ ]:
SIZE_NAME_BODY = 150
data_news['name_body'] = data_news['name_body'].apply(lambda x: ' '.join(x.split()[:SIZE_NAME_BODY]))

In [ ]:
data = pd.merge(data_broadcasts, data_news, on="match_id", how="inner")
data.rename(columns = {'name_x' : 'broadcast_name', 'name_y' : 'news_name'}, inplace = True)
data

,match_id,team1,team2,broadcast_name,match_time,concatenate_content,last_part_content,id,news_name,ctime,body,name_body,len_name_body
0,18925,Локомотив,Химки,Россия. Премьер-лига,1237037400,"Добрый день! Sports.ru поздравляет всех, кто п...",Драган Блатняк запустил мяч на трибуну со штра...,7274151,Дюрица получил травму в игре с «Химками»,2009-03-14 17:48:18,Защитник «Локомотива» Ян Дюрица получил травму...,Дюрица получил травму в игре с «Химками». Защи...,53
1,18925,Локомотив,Химки,Россия. Премьер-лига,1237037400,"Добрый день! Sports.ru поздравляет всех, кто п...",Драган Блатняк запустил мяч на трибуну со штра...,7274964,Антипенко получил травму в матче с «Локомотивом»,2009-03-14 19:12:30,Нападающий «Химок» Александр Антипенко получил...,Антипенко получил травму в матче с «Локомотиво...,47
2,18935,Зенит,Штутгарт,перенесенные матчи,1234977300,"Матчем со ""Штутгартом"" ""Зенит"" открывает европ...",Шибер впереди пока никак не зацепится за мяч. ...,7014478,Хусти забил в первом официальном матче в соста...,2009-02-18 20:36:18,Полузащитник «Зенита» Саболч Хусти отличился н...,Хусти забил в первом официальном матче в соста...,50
3,18956,Штутгарт,Зенит,перенесенные матчи,1235677500,"Ответный матч 1/16 финала Кубка УЕФА ""Зенита"" ...",Гомес толкнул Крижанаца в верховой борьбе. Нап...,7114149,Семшов забил первый гол в составе «Зенита»,2009-02-26 23:34:26,Полузащитник «Зенита» Игорь Семшов в матче 1/1...,Семшов забил первый гол в составе «Зенита». По...,59
4,18984,Химки,Динамо,Россия. Премьер-лига,1249210800,Привет! Второй матч воскресного дня ждет нас! ...,На спокойном зеркале игры вдруг наметилась тре...,22193831,Агьяр забил в дебютном матче в составе «Динамо»,2009-08-02 15:35:05,Новичок «Динамо» полузащитник Луис Агьяр забил...,Агьяр забил в дебютном матче в составе «Динамо...,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12833,1104865,Питтсбург,Сан-Хосе,Кубок Стэнли,1464652800,"Привет всем, кто не представляет своей жизни б...","""Сан Хосе"" не помогает и замена вратаря. Плюс ...",1040574809,«Питтсбург» – «Сан-Хосе». Раст может пропустит...,2016-05-31 08:52:37,Нападающий «Питтсбурга» Брайан Раст получил тр...,«Питтсбург» – «Сан-Хосе». Раст может пропустит...,115
12834,1104866,Питтсбург,Сан-Хосе,Кубок Стэнли,1464825600,"Привет всем, кто не представляет своей жизни б...","Совершенно фантастический вклад в победы ""Питт...",1040621229,НХЛ. Кубок Стэнли. Финал. «Питтсбург» вновь по...,2016-06-01 18:17:21,Во втором матче финала Кубка Стэнли «Питтсбург...,НХЛ. Кубок Стэнли. Финал. «Питтсбург» вновь по...,40
12835,1104866,Питтсбург,Сан-Хосе,Кубок Стэнли,1464825600,"Привет всем, кто не представляет своей жизни б...","Совершенно фантастический вклад в победы ""Питт...",1040636713,НХЛ. Форвард «Питтсбурга» Шери забил первый по...,2016-06-02 08:20:10,Во втором матче финала Кубка Стэнли «Питтсбург...,НХЛ. Форвард «Питтсбурга» Шери забил первый по...,86
12836,1104866,Питтсбург,Сан-Хосе,Кубок Стэнли,1464825600,"Привет всем, кто не представляет своей жизни б...","Совершенно фантастический вклад в победы ""Питт...",1040636981,НХЛ. Мюррей одержал 13-ю победу в дебютном пле...,2016-06-02 08:41:16,Во втором матче финала Кубка Стэнли «Питтсбург...,НХЛ. Мюррей одержал 13-ю победу в дебютном пле...,75


In [ ]:
good_news = []
bad_news = []
indexes = data.index.tolist()
random.shuffle(indexes)

for index in indexes:
  print('len(good_news)=', len(good_news))
  if len(good_news) == 750:
    break
  row = data.loc[index]
  print(row['news_name'])
  print('--------------------')
  pprint(row['body'], width=150)
  print('--------------------')
  pprint(row['last_part_content'], width=150)
  label = int(input())
  if label == 1:
    good_news.append(index)
  elif label == 0:
    bad_news.append(index)

  clear_output(True)

len(good_news)= 750


In [ ]:
len(bad_news)

324

In [ ]:
good_news

[12825,
 5495,
 8714,
 8093,
 11526,
 12576,
 741,
 7898,
 11567,
 7918,
 507,
 6118,
 10952,
 3927,
 10728,
 7716,
 12629,
 9157,
 9882,
 913,
 283,
 10993,
 12426,
 7895,
 10928,
 9581,
 74,
 9348,
 2486,
 8601,
 3445,
 4773,
 5379,
 9590,
 8610,
 4339,
 3278,
 7292,
 6213,
 3922,
 11880,
 9253,
 9336,
 4414,
 5819,
 142,
 5085,
 12077,
 11266,
 3546,
 8316,
 12615,
 5860,
 3111,
 10406,
 635,
 4868,
 5032,
 159,
 10255,
 12517,
 6542,
 5140,
 11354,
 6266,
 6227,
 4406,
 6619,
 9089,
 10453,
 8896,
 117,
 11897,
 4269,
 3447,
 10073,
 907,
 918,
 11114,
 7624,
 4694,
 5519,
 3247,
 4239,
 11510,
 7409,
 6345,
 10172,
 10439,
 7712,
 7470,
 5155,
 8004,
 10678,
 1548,
 10426,
 6039,
 6593,
 2286,
 2865,
 5469,
 9884,
 295,
 9728,
 2421,
 10743,
 9752,
 6512,
 1206,
 6135,
 11075,
 9058,
 9193,
 961,
 3655,
 8198,
 8730,
 6269,
 1542,
 5362,
 12187,
 5471,
 1520,
 2963,
 3144,
 5169,
 11204,
 1721,
 6203,
 1940,
 1951,
 4494,
 4546,
 9621,
 1453,
 5344,
 119,
 10906,
 5578,
 754,
 785

In [ ]:
bad_news

[3124,
 3931,
 2896,
 7997,
 2914,
 8278,
 6000,
 3258,
 9291,
 2652,
 12127,
 2278,
 8233,
 11076,
 9215,
 4760,
 547,
 2416,
 9799,
 4073,
 2534,
 12295,
 6600,
 6060,
 3609,
 629,
 10828,
 2736,
 10379,
 8766,
 4150,
 4852,
 10638,
 12655,
 10731,
 4794,
 12235,
 2850,
 1016,
 2668,
 100,
 3668,
 1307,
 3454,
 5843,
 1394,
 977,
 1403,
 1104,
 3299,
 7815,
 3757,
 11109,
 1179,
 10934,
 10680,
 290,
 4285,
 11660,
 1289,
 369,
 300,
 3715,
 9156,
 2522,
 11383,
 12433,
 2927,
 12255,
 9419,
 791,
 11875,
 4693,
 2408,
 12672,
 4966,
 9057,
 3565,
 9848,
 2516,
 2035,
 11568,
 3036,
 3602,
 8825,
 2883,
 6363,
 10736,
 8687,
 5775,
 12603,
 3293,
 4774,
 2541,
 4014,
 1291,
 3282,
 5812,
 3686,
 10768,
 8881,
 10930,
 4680,
 2402,
 2162,
 4417,
 2285,
 11955,
 11637,
 8808,
 7493,
 12656,
 7533,
 9644,
 1119,
 12759,
 5742,
 11845,
 10092,
 4393,
 11613,
 6383,
 1035,
 10101,
 9338,
 3407,
 2277,
 8082,
 8271,
 8850,
 5052,
 11564,
 9190,
 667,
 11432,
 82,
 737,
 170,
 171,
 1916,
 

In [ ]:
good_train = good_news[:500]
good_eval = good_news[500:]

bad_train = bad_news[:200]
bad_eval = bad_news[200:]

In [ ]:
data_good_train_result = data.loc[good_train]
data_good_train_result['result_match'] = 1

data_bad_train_result = data.loc[bad_train]
data_bad_train_result['result_match'] = 0

train = pd.concat([data_good_train_result, data_bad_train_result])
train = train.sample(frac=1, random_state=42)
train

,match_id,team1,team2,broadcast_name,match_time,concatenate_content,last_part_content,id,news_name,ctime,body,name_body,len_name_body,result_match
2472,455452,Дания,Германия,Чемпионат Европы,1339958700,Всем привет и добро пожаловать на ЕВРО-2012 бе...,Бендтнер зарабатывал этот угловой с таким осте...,141499717,Ларс Бендер забил первый гол за сборную Германии,2012-06-18 00:50:25,Полузащитник «Байера» и сборной Германии Ларс ...,Ларс Бендер забил первый гол за сборную Герман...,79,1
3124,476651,Бавария,Челси,Лига чемпионов,1337453100,Привет-привет! Настраиваемся? http://www.sport...,"Бить будут в ворота, за которыми мюнхенский су...",140901107,Дрогба сравнял счет в матче с «Баварией»,2012-05-20 00:34:22,Форвард «Челси» Дидье Дрогба забил гол в финал...,Дрогба сравнял счет в матче с «Баварией». Форв...,43,0
3704,481837,Кубань,Спартак,Россия. Премьер-лига,1347714000,"Здравствуйте, друзья. Продолжаем следить за со...","А меняет Прудников Кулика. ""Спартак"" ответил з...",143405274,«Кубань» – «Спартак» – 2:2. Озбилиз сравнял сч...,2012-09-15 12:57:47,Сегодня в Краснодаре в рамках 8-го тура РФПЛ м...,«Кубань» – «Спартак» – 2:2. Озбилиз сравнял сч...,94,1
12494,1089413,СКА,Локомотив,Кубок Гагарина,1456417800,"Привет всем, кто не представляет своей жизни б...",3:0 и 2-1 в серии - СКА одержал пока что свою ...,1037500549,"СКА – «Локомотив». Коскинен сыграл на ноль, от...",2016-02-25 22:21:05,Голкипер СКА Микко Коскинен сделал шатаут в тр...,"СКА – «Локомотив». Коскинен сыграл на ноль, от...",43,1
12578,1092758,СКА,Динамо Москва,Кубок Гагарина,1457877600,"Привет всем, кто не представляет своей жизни б...",Момент у СКА! Кетов после рывка Каблукова брос...,1038079919,СКА выиграл 5 матчей из 5 в плей-офф без Ковал...,2016-03-13 20:14:38,СКА обыграл московское «Динамо» (4:0) в четвер...,СКА выиграл 5 матчей из 5 в плей-офф без Ковал...,55,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,19212,Локомотив,Динамо,Россия. Премьер-лига,1248527700,Приветствуем Всех любителей самой популярной и...,"Сычев убежал по левому флангу, последовал прос...",20615828,Янбаев получил травму в матче с «Динамо»,2009-07-25 19:54:06,Защитник «Локомотива» Ренат Янбаев получил тра...,Янбаев получил травму в матче с «Динамо». Защи...,57,1
9752,982394,СКА,Металлург Нк,КХЛ,1417105800,"Привет всем, кто не представляет своей жизни б...","Бурдасов стремился вырваться на пятак, но был ...",1025295255,СКА – «Кузня» – 6:2. Кадейкин оформил дубль,2014-11-27 11:04:41,Сегодня СКА в матче регулярного чемпионата КХЛ...,СКА – «Кузня» – 6:2. Кадейкин оформил дубль. С...,48,1
466,210974,Салават Юлаев,Ак Барс,Кубок Гагарина,1270292400,"Привет-привет! Наточили зубы на Торесена? ""Сал...",Кто-то из казанцев выскользнул в одного защитн...,69957968,КХЛ. Плей-офф. 1/2 финала. Второй матч. «Ак Ба...,2010-04-03 11:14:56,Плей-офф 1/2 финала Второй матч Счет в серии: ...,КХЛ. Плей-офф. 1/2 финала. Второй матч. «Ак Ба...,28,1
6543,799070,Реал Мадрид,Вальядолид,Испания. Ла Лига,1385838000,"Салют, амигос! Как ""Реал"" будет играть без Арб...",Пока же Манушу меняет Хави Герру. А у хозяев Б...,154751298,"Бэйл стал вторым британцем в истории примеры, ...",2013-12-01 02:29:54,Полузащитник «Реала» Гарет Бэйл забил три гола...,"Бэйл стал вторым британцем в истории примеры, ...",83,1


In [ ]:
# добавочные новости для train. пул новостей при активном обучении
indexes = data.index.tolist()
random.shuffle(indexes)

add_news = []

for index in indexes:
  if index not in good_news and index not in bad_news:
    add_news.append(index)

In [ ]:
data_add_train_result = data.loc[add_news]
data_add_train_result['result_match'] = 0


pool_train = pd.concat([train, data_add_train_result])
pool_train

,match_id,team1,team2,broadcast_name,match_time,concatenate_content,last_part_content,id,news_name,ctime,body,name_body,len_name_body,result_match
2472,455452,Дания,Германия,Чемпионат Европы,1339958700,Всем привет и добро пожаловать на ЕВРО-2012 бе...,Бендтнер зарабатывал этот угловой с таким осте...,141499717,Ларс Бендер забил первый гол за сборную Германии,2012-06-18 00:50:25,Полузащитник «Байера» и сборной Германии Ларс ...,Ларс Бендер забил первый гол за сборную Герман...,79,1
3124,476651,Бавария,Челси,Лига чемпионов,1337453100,Привет-привет! Настраиваемся? http://www.sport...,"Бить будут в ворота, за которыми мюнхенский су...",140901107,Дрогба сравнял счет в матче с «Баварией»,2012-05-20 00:34:22,Форвард «Челси» Дидье Дрогба забил гол в финал...,Дрогба сравнял счет в матче с «Баварией». Форв...,43,0
3704,481837,Кубань,Спартак,Россия. Премьер-лига,1347714000,"Здравствуйте, друзья. Продолжаем следить за со...","А меняет Прудников Кулика. ""Спартак"" ответил з...",143405274,«Кубань» – «Спартак» – 2:2. Озбилиз сравнял сч...,2012-09-15 12:57:47,Сегодня в Краснодаре в рамках 8-го тура РФПЛ м...,«Кубань» – «Спартак» – 2:2. Озбилиз сравнял сч...,94,1
12494,1089413,СКА,Локомотив,Кубок Гагарина,1456417800,"Привет всем, кто не представляет своей жизни б...",3:0 и 2-1 в серии - СКА одержал пока что свою ...,1037500549,"СКА – «Локомотив». Коскинен сыграл на ноль, от...",2016-02-25 22:21:05,Голкипер СКА Микко Коскинен сделал шатаут в тр...,"СКА – «Локомотив». Коскинен сыграл на ноль, от...",43,1
12578,1092758,СКА,Динамо Москва,Кубок Гагарина,1457877600,"Привет всем, кто не представляет своей жизни б...",Момент у СКА! Кетов после рывка Каблукова брос...,1038079919,СКА выиграл 5 матчей из 5 в плей-офф без Ковал...,2016-03-13 20:14:38,СКА обыграл московское «Динамо» (4:0) в четвер...,СКА выиграл 5 матчей из 5 в плей-офф без Ковал...,55,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,32449,Чили,Швейцария,перенесенные матчи,1277128800,"Привет! Еще чуть-чуть, и мой тезка даст свисто...","Эх, Гонсалес... Шел по центру, справа было два...",71365938,Швейцарец Бехрами удален в первом тайме матча ...,2010-06-21 18:37:37,Полузащитник сборной Швейцарии Валон Бехрами в...,Швейцарец Бехрами удален в первом тайме матча ...,76,0
1562,293545,Россия,Словакия,Чемпионат мира,1304446500,Вкус детских качелек все никак не дает покоя? ...,Польские комментаторы словно уговаривают Мариа...,94961248,Шатан сравнял счет в матче с Россией,2011-05-03 22:23:06,Форвард сборной Словакии Мирослав Шатан сравня...,Шатан сравнял счет в матче с Россией. Форвард ...,59,0
12326,1077519,Уфа,ЦСКА,Россия. Олимп Кубок России,1456840800,"Всем привет! Пришла весна, и вернулся российск...",Третья замена в ЦСКА : Аарон Олонаре вместо Ах...,1037645023,Алексей Березуцкий стал рекордсменом ЦСКА по ч...,2016-03-01 17:26:23,Защитник ЦСКА Алексей Березуцкий установил клу...,Алексей Березуцкий стал рекордсменом ЦСКА по ч...,83,0
562,218578,Дания,Россия,Чемпионат мира,1274019300,"Привет! Так, кто там у нас на очереди? Дания! ...","Ох, не прошел пас Семина на дальнюю штангу, гд...",70695787,Сборная России вышла в четвертьфинал чемпионат...,2010-05-16 01:42:01,Сборная России после победы над Германией (3:2...,Сборная России вышла в четвертьфинал чемпионат...,87,0


In [ ]:
data_good_eval_result = data.loc[good_eval]
data_good_eval_result['result_match'] = 1

data_bad_eval_result = data.loc[bad_eval]
data_bad_eval_result['result_match'] = 0

eval = pd.concat([data_good_eval_result, data_bad_eval_result])
eval = eval.sample(frac=1, random_state=42)
eval

,match_id,team1,team2,broadcast_name,match_time,concatenate_content,last_part_content,id,news_name,ctime,body,name_body,len_name_body,result_match
10056,991265,Шахтер,БАТЭ,Лига чемпионов,1415216700,Всем привет! В рамках четвертого тура группово...,Бернард поменял Тайсона. Обычно гимн Украины б...,1024772242,"Луис Адриано стал первым игроком, которому уда...",2014-11-06 01:05:10,Форвард донецкого «Шахтера» Луис Адриано устан...,"Луис Адриано стал первым игроком, которому уда...",54,0
553,217803,Сан-Хосе,Детройт,Кубок Стэнли,1273370400,"Привет всем, ко жить не может без НХЛ! Sports....",Отличный сэйв Ховарда! Никол атаковал в упор с...,70570517,НХЛ. «Сан-Хосе» второй раз в истории вышел в ф...,2010-05-09 09:22:09,Калифорнийский клуб «Сан-Хосе» после победы на...,НХЛ. «Сан-Хосе» второй раз в истории вышел в ф...,114,1
11474,1035331,Кубань,Локомотив,Россия. Премьер-лига,1446393600,"Всем привет. Добро пожаловать в Краснодар, где...","Все, завершен матч . ""Кубань"" просто уничтожил...",1034282031,«Кубань» – «Локомотив» – 6:2. Железнодорожники...,2015-11-01 09:29:49,Сегодня в Краснодаре в матче 14-го тура РФПЛ «...,«Кубань» – «Локомотив» – 6:2. Железнодорожники...,55,1
7394,830015,Германия,Гана,Чемпионат мира,1403377200,"Сегодня в Форталезе на арене ""Кастелао"", где у...",Швайнштайгер прессует атаки немцев глубже в не...,1021554058,Германия – Гана. Гетце забил гол,2014-06-22 00:05:23,Атакующий полузащитник сборной Германии Марио ...,Германия – Гана. Гетце забил гол. Атакующий по...,76,0
9502,966246,Барселона,Реал Сосьедад,Испания. Ла Лига,1431187200,Всем привет! Большой футбольный уик-энд набира...,"Очередная замена: Рафинья уходит, Педро выходи...",1029562083,«Барселона» – «Реал Сосьедад» – 2:0. Неймар за...,2015-05-09 15:22:51,В 36-м туре чемпионата Испании «Барселона» обы...,«Барселона» – «Реал Сосьедад» – 2:0. Неймар за...,34,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6950,815435,СКА,Авангард,КХЛ,1393428600,"Привет всем, кто не представляет своей жизни б...",Отмечу прекрасную статистику Ружички после пер...,158241356,СКА – «Авангард» – 4:5 ОТ. Широков принес побе...,2014-02-26 19:31:03,Сегодня в Петербурге проходит матч регулярного...,СКА – «Авангард» – 4:5 ОТ. Широков принес побе...,31,1
10365,1013451,Барыс,Авангард,Кубок Гагарина,1425389400,Всем привет! Сегодня в Кубке Гагарина большой ...,"Хозяева, как и во втором матче, переводят игру...",1027691296,«Барыс» – «Авангард» – 3:4. Курьянов забросил ...,2015-03-04 14:39:39,Сегодня в четвертой игре серии первого раунда ...,«Барыс» – «Авангард» – 3:4. Курьянов забросил ...,40,1
6538,799065,Атлетик,Барселона,Испания. Ла Лига,1385928000,"Салют, амигос! ""Атлетик"" - ""Барселона"" - проти...",Чуть было сами себе не привезли хозяева! Оступ...,154776181,«Атлетик» – «Барселона». Пике проводит 150-й м...,2013-12-02 00:24:55,Защитник «Барселоны» Жерар Пике вышел в старто...,«Атлетик» – «Барселона». Пике проводит 150-й м...,71,0
5865,786385,Манчестер Юнайтед,Халл,Англия. Премьер-лига,1399401900,"В последних трех матчах АПЛ ""Халл"" набрал лишь...","Феллайни совсем не в склад, не в лад. Кагава е...",1020359093,«Манчестер Юнайтед» – «Халл». Джонс был госпит...,2014-05-07 00:23:57,"Защитник «Манчестер Юнайтед» Фил Джонс, получи...",«Манчестер Юнайтед» – «Халл». Джонс был госпит...,79,0


In [ ]:
pool_train.to_csv('train.csv', index=False, encoding='utf-8')

In [ ]:
!cp /content/train.csv /content/drive/MyDrive/RuSportSum/train_2.csv

In [ ]:
eval.to_csv('eval.csv', index=False, encoding='utf-8')

In [ ]:
!cp /content/eval.csv /content/drive/MyDrive/RuSportSum/eval_2.csv